In [3]:
import pandas as pd
import joblib

# Load the joblib ADS-B file
adsb_data = joblib.load("data_2022_july.joblib")
adsb_df = adsb_data.data
adsb_df.head()


,timestamp,icao24,latitude,longitude,groundspeed,track,vertical_rate,callsign,onground,alert,...,squawk,altitude,geoaltitude,last_position,hour,firstseen,origin,lastseen,destination,day
0,2022-06-30 22:00:01+00:00,505d14,51.249638,10.823135,406.0,338.617948,-1920.0,CXI2048,False,False,...,1000,24450.0,25475.0,2022-06-30 21:59:58.956000+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00
1,2022-06-30 22:00:02+00:00,505d14,51.249638,10.823135,406.0,338.617948,-1920.0,CXI2048,False,False,...,1000,24400.0,25475.0,2022-06-30 21:59:58.956000+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00
2,2022-06-30 22:00:03+00:00,505d14,51.255829,10.819321,405.0,338.566371,-1920.0,CXI2048,False,False,...,1000,24375.0,25425.0,2022-06-30 22:00:02.688999936+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00
3,2022-06-30 22:00:04+00:00,505d14,51.257924,10.818024,406.0,338.749494,-1920.0,CXI2048,False,False,...,1000,24350.0,25400.0,2022-06-30 22:00:03.819000064+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00
4,2022-06-30 22:00:05+00:00,505d14,51.260205,10.816574,405.0,338.566371,-1984.0,CXI2048,False,False,...,1000,24325.0,25375.0,2022-06-30 22:00:04.990000128+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00


In [4]:
from traffic.data import aircraft

#  Ensure all ICAO24 identifiers are lowercase
adsb_df["icao24"] = adsb_df["icao24"].str.lower()

# Extract unique icao24s (faster than looping over full dataframe)
unique_icao24 = adsb_df["icao24"].dropna().unique()


# A unique flight identifier by grouping by (icao24, date)
adsb_df["flight_id"] = adsb_df.groupby(["icao24", "day"]).ngroup()

# Build a lookup dictionary with typecodes
def safe_get_typecode(icao):
    info = aircraft.get(icao)
    return getattr(info, "typecode", None) if info else None

icao_to_typecode = {icao: safe_get_typecode(icao) for icao in unique_icao24}

# Mapping typecodes back to the original dataframe
adsb_df["typecode"] = adsb_df["icao24"].map(icao_to_typecode)

# Done — preview result
adsb_df[["icao24", "typecode"]].drop_duplicates().head()


,icao24,typecode
0,505d14,B738
1051,30019c,B738
4186,4bb846,A20N
7785,3c0cb2,B738
9800,440953,A320


In [5]:
# Sort by time (to maintain trajectory order)
adsb_df.groupby(["icao24","day"]).timestamp.min()

icao24  day                      
0100e4  2022-07-27 00:00:00+00:00   2022-07-27 20:43:57+00:00
        2022-07-28 00:00:00+00:00   2022-07-28 01:30:51+00:00
0100e5  2022-07-16 00:00:00+00:00   2022-07-16 20:33:30+00:00
        2022-07-17 00:00:00+00:00   2022-07-17 01:54:28+00:00
        2022-07-24 00:00:00+00:00   2022-07-23 21:26:51+00:00
                                               ...           
ad7a5f  2022-07-07 00:00:00+00:00   2022-07-07 02:05:44+00:00
        2022-07-08 00:00:00+00:00   2022-07-08 01:59:45+00:00
        2022-07-11 00:00:00+00:00   2022-07-11 21:00:25+00:00
ad8b94  2022-07-27 00:00:00+00:00   2022-07-27 02:26:33+00:00
ad9302  2022-07-05 00:00:00+00:00   2022-07-05 02:21:24+00:00
Name: timestamp, Length: 2538, dtype: datetime64[ns, UTC]

In [7]:
adsb_df.head()

,timestamp,icao24,latitude,longitude,groundspeed,track,vertical_rate,callsign,onground,alert,...,altitude,geoaltitude,last_position,hour,firstseen,origin,lastseen,destination,day,typecode
0,2022-06-30 22:00:01+00:00,505d14,51.249638,10.823135,406.0,338.617948,-1920.0,CXI2048,False,False,...,24450.0,25475.0,2022-06-30 21:59:58.956000+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00,B738
1,2022-06-30 22:00:02+00:00,505d14,51.249638,10.823135,406.0,338.617948,-1920.0,CXI2048,False,False,...,24400.0,25475.0,2022-06-30 21:59:58.956000+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00,B738
2,2022-06-30 22:00:03+00:00,505d14,51.255829,10.819321,405.0,338.566371,-1920.0,CXI2048,False,False,...,24375.0,25425.0,2022-06-30 22:00:02.688999936+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00,B738
3,2022-06-30 22:00:04+00:00,505d14,51.257924,10.818024,406.0,338.749494,-1920.0,CXI2048,False,False,...,24350.0,25400.0,2022-06-30 22:00:03.819000064+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00,B738
4,2022-06-30 22:00:05+00:00,505d14,51.260205,10.816574,405.0,338.566371,-1984.0,CXI2048,False,False,...,24325.0,25375.0,2022-06-30 22:00:04.990000128+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00,B738


In [8]:
df_noise = pd.read_csv("Noise_df.csv")

# Rename and clean columns for consistency
df_noise.rename(columns={
    "ATA/ATD": "noise_time",
    "A/D": "phase",
    "TLASmax": "TLASmax",
    "Abstand [m]": "distance_m",
    "Höhenwinkel [°]": "angle_deg",
    "Höhe [ft]": "altitude_ft",
    "T10 [s]": "T10",
    "LAE": "LAE"
}, inplace=True)

# Convert time to datetime
df_noise["noise_time"] = pd.to_datetime(df_noise["noise_time"])

In [9]:
display(df_noise)

,MP,noise_time,phase,Runway,SID/STAR,Flugzeugtyp,MTOM,Triebwerk,TLASmax,distance_m,angle_deg,altitude_ft,LASmax,Leq,LAE,T10,TGesamt [s],AzB-Klasse
0,M 01,2022-01-01 07:52:10+00:00,Landung,27L,NaN,E290,54000.0,S,01.01.2022 08:51:12,315.0,34.2,761.0,74.4,70.4,84.0,18.8,23,S 5.2
1,M 01,2022-01-01 08:04:09+00:00,Landung,27L,NaN,A320,78000.0,S,01.01.2022 09:03:03,270.0,39.9,749.0,80.2,75.1,89.1,14.6,25,S 5.2
2,M 01,2022-01-01 09:39:42+00:00,Landung,27L,NaN,E170,33990.0,S,01.01.2022 10:38:25,203.0,69.8,805.0,76.0,72.0,86.1,19.2,26,S 5.1
3,M 01,2022-01-01 11:40:16+00:00,Landung,27L,NaN,A321,89000.0,S,01.01.2022 12:39:29,196.0,67.6,775.0,77.1,72.1,85.9,15.1,24,S 5.2
4,M 01,2022-01-01 13:17:08+00:00,Landung,27L,NaN,CRJ9,37990.0,S,01.01.2022 14:15:55,248.0,50.4,807.0,77.1,72.2,85.4,14.2,21,S 5.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92633,M 09,2022-12-31 14:16:59+00:00,Start,27R,POVEL3S,A320,78000.0,S,31.12.2022 15:17:54,597.0,79.9,2092.0,77.4,71.4,87.2,24.0,38,S 5.2
92634,M 09,2022-12-31 15:43:29+00:00,Start,27R,VAXEV1S,A320,77000.0,S,31.12.2022 16:44:02,493.0,74.0,1719.0,79.1,73.7,88.6,22.0,31,S 5.2
92635,M 09,2022-12-31 17:18:41+00:00,Start,27R,VAXEV1S,E190,45000.0,S,31.12.2022 18:19:56,1020.0,55.0,2905.0,75.7,70.0,85.4,27.8,35,S 5.2
92636,M 09,2022-12-31 17:21:01+00:00,Start,27L,POVEL3F,A321,89000.0,S,31.12.2022 18:22:09,1736.0,41.5,3938.0,71.1,65.8,82.5,44.3,47,S 5.2


In [10]:
import pandas as pd

# Convert timestamps to datetime
adsb_df['timestamp'] = pd.to_datetime(adsb_df['timestamp'], unit='s')
df_noise['noise_time'] = pd.to_datetime(df_noise['noise_time'], errors='coerce')

# Filtering both DataFrames for valid timestamps and type info
adsb_df = adsb_df.dropna(subset=['timestamp', 'typecode'])
df_noise = df_noise.dropna(subset=['noise_time', 'Flugzeugtyp'])

# Sorting both DataFrames for merge_asof
adsb_df = adsb_df.sort_values('timestamp')
df_noise = df_noise.sort_values('noise_time')


adsb_df = adsb_df.rename(columns={'typecode': 'Flugzeugtyp'})  # for merge compatibility

# merge_asof with both time and aircraft type filtering
# Performing it group-wise by 'Flugzeugtyp' to match only within the same type
matched_rows = []

for flz_type in df_noise['Flugzeugtyp'].unique():
    adsb_sub = adsb_df[adsb_df['Flugzeugtyp'] == flz_type]
    noise_sub = df_noise[df_noise['Flugzeugtyp'] == flz_type]
    
    if not adsb_sub.empty and not noise_sub.empty:
        merged = pd.merge_asof(
            noise_sub.sort_values('noise_time'),
            adsb_sub.sort_values('timestamp'),
            left_on='noise_time',
            right_on='timestamp',
            direction='nearest',
            tolerance=pd.Timedelta('10m')
        )
        matched_rows.append(merged)

# Combine all matches
combined_df = pd.concat(matched_rows, ignore_index=True)

# Drop rows with no match
combined_df = combined_df.dropna(subset=['icao24'])

# Rename 'Flugzeugtyp' back if needed
combined_df = combined_df.rename(columns={'Flugzeugtyp': 'typecode'})

# Final preview
combined_df.head()


,MP,noise_time,phase,Runway,SID/STAR,Flugzeugtyp_x,MTOM,Triebwerk,TLASmax,distance_m,...,altitude,geoaltitude,last_position,hour,firstseen,origin,lastseen,destination,day,Flugzeugtyp_y
16142,M 03,2022-06-30 22:07:08+00:00,Landung,27R,NaN,B738,79015.0,S,01.07.2022 00:05:38,430.0,...,15100.0,15775.0,2022-06-30 22:07:07.678999808+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00,B738
16143,M 05,2022-06-30 22:07:08+00:00,Landung,27R,NaN,B738,79015.0,S,01.07.2022 00:04:59,602.0,...,15100.0,15775.0,2022-06-30 22:07:07.678999808+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00,B738
16144,M 02,2022-06-30 22:07:08+00:00,Landung,27R,NaN,B738,79015.0,S,01.07.2022 00:06:11,233.0,...,15100.0,15775.0,2022-06-30 22:07:07.678999808+00:00,2022-06-30 22:00:00+00:00,2022-06-30 19:09:40+00:00,NaN,2022-06-30 22:17:31+00:00,EDDV,2022-06-30 00:00:00+00:00,B738
16145,M 05,2022-06-30 22:17:48+00:00,Landung,27R,NaN,B738,79015.0,S,01.07.2022 00:15:39,768.0,...,36000.0,37725.0,2022-06-30 22:17:47.876000+00:00,2022-06-30 22:00:00+00:00,2022-06-30 21:48:43+00:00,NaN,2022-07-01 00:27:45+00:00,EDDV,2022-07-01 00:00:00+00:00,B738
16146,M 02,2022-06-30 22:17:48+00:00,Landung,27R,NaN,B738,79015.0,S,01.07.2022 00:16:54,349.0,...,36000.0,37725.0,2022-06-30 22:17:47.876000+00:00,2022-06-30 22:00:00+00:00,2022-06-30 21:48:43+00:00,NaN,2022-07-01 00:27:45+00:00,EDDV,2022-07-01 00:00:00+00:00,B738


In [11]:
combined_df.columns

Index(['MP', 'noise_time', 'phase', 'Runway', 'SID/STAR', 'Flugzeugtyp_x',
       'MTOM', 'Triebwerk', 'TLASmax', 'distance_m', 'angle_deg',
       'altitude_ft', 'LASmax', 'Leq', 'LAE', 'T10', 'TGesamt [s]',
       'AzB-Klasse', 'timestamp', 'icao24', 'latitude', 'longitude',
       'groundspeed', 'track', 'vertical_rate', 'callsign', 'onground',
       'alert', 'spi', 'squawk', 'altitude', 'geoaltitude', 'last_position',
       'hour', 'firstseen', 'origin', 'lastseen', 'destination', 'day',
       'Flugzeugtyp_y'],
      dtype='object')

In [12]:
combined_df[['noise_time', 'timestamp']]

,noise_time,timestamp
16142,2022-06-30 22:07:08+00:00,2022-06-30 22:07:08+00:00
16143,2022-06-30 22:07:08+00:00,2022-06-30 22:07:08+00:00
16144,2022-06-30 22:07:08+00:00,2022-06-30 22:07:08+00:00
16145,2022-06-30 22:17:48+00:00,2022-06-30 22:17:48+00:00
16146,2022-06-30 22:17:48+00:00,2022-06-30 22:17:48+00:00
...,...,...
89837,2022-07-30 23:44:11+00:00,2022-07-30 23:43:42+00:00
89838,2022-07-30 23:44:11+00:00,2022-07-30 23:43:42+00:00
89839,2022-07-30 23:44:11+00:00,2022-07-30 23:43:42+00:00
89840,2022-07-31 02:14:29+00:00,2022-07-31 02:14:29+00:00


In [13]:
combined_df.to_csv(r'D:\Braunschweig\Course\Semester_2\HIWI_1\Work\AircraftNoiseTask\Machine Learning NN\Data\Matched_data_July')